<a href="https://colab.research.google.com/github/psaraswatshell/AiAgile/blob/master/XLNET_ZEMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Apr 27 22:41:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!rm -rf apex
!git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6593 (delta 0), reused 2 (delta 0), pack-reused 6589
Receiving objects: 100% (6593/6593), 13.71 MiB | 27.41 MiB/s, done.
Resolving deltas: 100% (4384/4384), done.


In [3]:
!cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-7a8n92wg
Created temporary directory: /tmp/pip-req-tracker-n7qrf6sm
Created requirements tracker '/tmp/pip-req-tracker-n7qrf6sm'
Created temporary directory: /tmp/pip-install-mdau65pi
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-da6cb7r4
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-n7qrf6sm'
    Running setup.py (path:/tmp/pip-req-build-da6cb7r4/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.4.0
    running egg_info
    creating /tmp/pip-req-build-da6cb7r4/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-da6cb7r4/pip-egg-info/apex.egg-info/PKG-INFO
  

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!pip install -r './drive/My Drive/Colab Notebooks/requirements.txt'

     |████████████████████████████████| 204kB 4.4MB/s 
     |████████████████████████████████| 573kB 9.1MB/s 
     |████████████████████████████████| 3.8MB 13.1MB/s 
     |████████████████████████████████| 1.0MB 44.2MB/s 
     |████████████████████████████████| 890kB 53.9MB/s 
     |████████████████████████████████| 112kB 45.8MB/s 
     |████████████████████████████████| 552kB 53.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=c93d669b0e63c5fbe524bd51ff36abb050097e239ffc0952d3ac53b61f1aed71
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
ERROR: transformers 2.8.0 has requirement tokenizers==0.5.2, but you'll have tokenizers 0.7.0 which is incompatible.


In [6]:
!pip install fast-bert

     |████████████████████████████████| 81kB 3.7MB/s 


In [0]:
from transformers import BertTokenizer
from pathlib import Path
import torch

from box import Box
import pandas as pd
import collections
import os
from tqdm import tqdm, trange
import sys
import random
import numpy as np
import apex
from sklearn.model_selection import train_test_split

import datetime

from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc

In [0]:
torch.cuda.empty_cache()

In [9]:
pd.set_option('display.max_colwidth', -1)
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [0]:
DATA_PATH = Path('./drive/My Drive/Colab Notebooks/ap_product/')
LABEL_PATH = Path('./drive/My Drive/Colab Notebooks/ap_product/')

AUG_DATA_PATH = Path('./drive/My Drive/Colab Notebooks/ap_product/data-aug/')

MODEL_PATH=Path('./drive/My Drive/Colab Notebooks/ap_product/models')
LOG_PATH=Path('./drive/My Drive/Colab Notebooks/ap_product/logs/')
MODEL_PATH.mkdir(exist_ok=True)

model_state_dict = None

# BERT_PRETRAINED_PATH = Path('../../bert_models/pretrained-weights/cased_L-12_H-768_A-12/')
BERT_PRETRAINED_PATH = Path('/content/bert_models/pretrained-weights/uncased_L-12_H-768_A-12/')
# BERT_PRETRAINED_PATH = Path('../../bert_fastai/pretrained-weights/uncased_L-24_H-1024_A-16/')
# FINETUNED_PATH = Path('../models/finetuned_model.bin')
FINETUNED_PATH = None
# model_state_dict = torch.load(FINETUNED_PATH)

LOG_PATH.mkdir(exist_ok=True)

OUTPUT_PATH = MODEL_PATH/'output'
OUTPUT_PATH.mkdir(exist_ok=True)

In [0]:
args = Box({
    "run_text": "multilabel toxic comments with freezable layers",
    "train_size": -1,
    "val_size": -1,
    "log_path": LOG_PATH,
    "full_data_dir": DATA_PATH,
    "data_dir": DATA_PATH,
    "task_name": "product_classsification_lib",
    "no_cuda": False,
    "bert_model": BERT_PRETRAINED_PATH,
    "output_dir": OUTPUT_PATH,
    "max_seq_length": 512,
    "do_train": True,
    "do_eval": True,
    "do_lower_case": True,
    "train_batch_size": 8,
    "eval_batch_size": 16,
    "learning_rate": 5e-5,
    "num_train_epochs": 6,
    "warmup_proportion": 0.0,
    "no_cuda": False,
    "local_rank": -1,
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "optimize_on_cpu": False,
    "fp16": True,
    "fp16_opt_level": "O1",
    "weight_decay": 0.0,
    "adam_epsilon": 1e-8,
    "max_grad_norm": 1.0,
    "max_steps": -1,
    "warmup_steps": 500,
    "logging_steps": 50,
    "eval_all_checkpoints": True,
    "overwrite_output_dir": True,
    "overwrite_cache": False,
    "seed": 42,
    "loss_scale": 128,
    "task_name": 'intent',
    "model_name": 'xlnet-base-cased',
    "model_type": 'xlnet'
})

In [0]:
import logging

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

In [13]:
logger.info(args)

04/27/2020 22:51:37 - INFO - root -   {'run_text': 'multilabel toxic comments with freezable layers', 'train_size': -1, 'val_size': -1, 'log_path': PosixPath('drive/My Drive/Colab Notebooks/ap_product/logs'), 'full_data_dir': PosixPath('drive/My Drive/Colab Notebooks/ap_product'), 'data_dir': PosixPath('drive/My Drive/Colab Notebooks/ap_product'), 'task_name': 'intent', 'no_cuda': False, 'bert_model': PosixPath('/content/bert_models/pretrained-weights/uncased_L-12_H-768_A-12'), 'output_dir': PosixPath('drive/My Drive/Colab Notebooks/ap_product/models/output'), 'max_seq_length': 512, 'do_train': True, 'do_eval': True, 'do_lower_case': True, 'train_batch_size': 8, 'eval_batch_size': 16, 'learning_rate': 5e-05, 'num_train_epochs': 6, 'warmup_proportion': 0.0, 'local_rank': -1, 'seed': 42, 'gradient_accumulation_steps': 1, 'optimize_on_cpu': False, 'fp16': True, 'fp16_opt_level': 'O1', 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'max_steps': -1, 'warmup_steps': 500, '

In [0]:
device = torch.device('cuda')
if torch.cuda.device_count() > 1:
    args.multi_gpu = True
else:
    args.multi_gpu = False


In [0]:
label_cols = ["Natural Gas", "Crude Oil", "Diesel", "Gasoline", "Coal", "Propane", "Fuel Oil", "PropaneButane", "Base Oil", "Benzene", "Styrene", "Butadiene", "Propylene", "Gasoil", "Isobutylene", "Polybutadiene", "Acrylonitrite", "Ethylene", "Phenol", "Cyclohexane", "Ehtylene", "Mogas", "Jet Fuel", "Butane", "Isobutane", "Rig Count", "Naphtha", "VGO", "Cumene", "Ethane", "EthanePropane", "Marine Fuel"]

In [16]:
databunch = BertDataBunch(args['data_dir'], LABEL_PATH, args.model_name, train_file='train.csv', val_file='val.csv',
                          test_data='test.csv',
                          text_col="Text", label_col=label_cols,
                          batch_size_per_gpu=args['train_batch_size'], max_seq_length=args['max_seq_length'], 
                          multi_gpu=args.multi_gpu, multi_label=True, model_type=args.model_type)

04/27/2020 22:51:40 - INFO - filelock -   Lock 139888439244952 acquired on /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.f23f7137b19a096a6eed89d1ffd00d4530935d72381eb7b9fcf8d5a1f25919ad.lock
04/27/2020 22:51:40 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpx60roh83



04/27/2020 22:51:41 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json in cache at /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.f23f7137b19a096a6eed89d1ffd00d4530935d72381eb7b9fcf8d5a1f25919ad
04/27/2020 22:51:41 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.f23f7137b19a096a6eed89d1ffd00d4530935d72381eb7b9fcf8d5a1f25919ad
04/27/2020 22:51:41 - INFO - filelock -   Lock 139888439244952 released on /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.f23f7137b19a096a6eed89d1ffd00d4530935d72381eb7b9fcf8d5a1f25919ad.lock
04/27/2020 22:51:41 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json from cache at /ro


04/27/2020 22:51:42 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model in cache at /root/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8
04/27/2020 22:51:42 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8
04/27/2020 22:51:42 - INFO - filelock -   Lock 139886978506032 released on /root/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8.lock
04/27/2020 22:51:42 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-spiece.model from cache at /root/.cache/tor

In [17]:
databunch.train_dl.dataset[0][3]

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
# train_df.head(20)
# databunch = BertDataBunch.load(args['data_dir'])

In [19]:
num_labels = len(databunch.labels)
num_labels

32

In [0]:
# torch.distributed.init_process_group(backend="nccl", 
#                                      init_method = "tcp://localhost:23459", 
#                                      rank=0, world_size=1)

In [0]:
metrics = []
metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'fbeta', 'function': fbeta})

In [22]:
learner = BertLearner.from_pretrained_model(databunch, args.model_name, metrics=metrics, 
                                            device=device, logger=logger, output_dir=args.output_dir, 
                                            finetuned_wgts_path=FINETUNED_PATH, warmup_steps=args.warmup_steps,
                                            multi_gpu=args.multi_gpu, is_fp16=args.fp16, 
                                            multi_label=True, logging_steps=0)



04/27/2020 22:51:51 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json from cache at /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.f23f7137b19a096a6eed89d1ffd00d4530935d72381eb7b9fcf8d5a1f25919ad
04/27/2020 22:51:51 - INFO - transformers.configuration_utils -   Model config XLNetConfig {
  "_num_labels": 32,
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bad_words_ids": null,
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "decoder_start_token_id": null,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": false,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"


04/27/2020 22:52:04 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-pytorch_model.bin in cache at /root/.cache/torch/transformers/24197ba0ce5dbfe23924431610704c88e2c0371afa49149360e4c823219ab474.7eac4fe898a021204e63c88c00ea68c60443c57f94b4bc3c02adbde6465745ac
04/27/2020 22:52:04 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/24197ba0ce5dbfe23924431610704c88e2c0371afa49149360e4c823219ab474.7eac4fe898a021204e63c88c00ea68c60443c57f94b4bc3c02adbde6465745ac
04/27/2020 22:52:04 - INFO - filelock -   Lock 139891010117584 released on /root/.cache/torch/transformers/24197ba0ce5dbfe23924431610704c88e2c0371afa49149360e4c823219ab474.7eac4fe898a021204e63c88c00ea68c60443c57f94b4bc3c02adbde6465745ac.lock
04/27/2020 22:52:04 - INFO - transformers.modeling_utils -   loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-pytorch_model.bin from cache at /r

In [23]:
!nvidia-smi

Mon Apr 27 22:52:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    32W / 250W |   1303MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [31]:
learner.fit(args.num_train_epochs, args.learning_rate, validate=True)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
04/27/2020 18:15:05 - INFO - root -   ***** Running training *****
04/27/2020 18:15:05 - INFO - root -     Num examples = 8703
04/27/2020 18:15:05 - INFO - root -     Num Epochs = 6
04/27/2020 18:15:05 - INFO - root -     Total train batch size (w. parallel, distributed & accumulation) = 8
04/27/2020 18:15:05 - 

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


04/27/2020 18:33:26 - INFO - root -   Running evaluation
04/27/2020 18:33:26 - INFO - root -     Num examples = 2176
04/27/2020 18:33:26 - INFO - root -     Batch size = 16


04/27/2020 18:35:17 - INFO - root -   eval_loss after epoch 1: 0.11682061763370738: 
04/27/2020 18:35:17 - INFO - root -   eval_accuracy_thresh after epoch 1: 0.9889849424362183: 
04/27/2020 18:35:17 - INFO - root -   eval_roc_auc after epoch 1: 0.9525962273501595: 
04/27/2020 18:35:17 - INFO - root -   eval_fbeta after epoch 1: 0.6548713445663452: 
04/27/2020 18:35:17 - INFO - root -   lr after epoch 1: 4.883529329914372e-05
04/27/2020 18:35:17 - INFO - root -   train_loss after epoch 1: 0.3106657691532746
04/27/2020 18:35:17 - INFO - root -   



/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

04/27/2020 19:15:13 - INFO - root -   eval_loss after epoch 3: 0.04067492339869633: 
04/27/2020 19:15:13 - INFO - root -   eval_accuracy_thresh after epoch 3: 0.99658203125: 
04/27/2020 19:15:13 - INFO - root -   eval_roc_auc after epoch 3: 0.9945829646743586: 
04/27/2020 19:15:13 - INFO - root -   eval_fbeta after epoch 3: 0.8979779481887817: 
04/27/2020 19:15:13 - INFO - root -   lr after epoch 3: 2.8248083552905847e-05
04/27/2020 19:15:13 - INFO - root -   train_loss after epoch 3: 0.053735122925204715
04/27/2020 19:15:13 - INFO - root -   

04/27/2020 19:33:23 - INFO - root -   Running evaluation
04/27/2020 19:33:23 - INFO - root -     Num examples = 2176
04/27/2020 19:33:23 - INFO - root -     Batch size = 16


04/27/2020 19:35:13 - INFO - root -   eval_loss after epoch 4: 0.03191018425037756: 
04/27/2020 19:35:13 - INFO - root -   eval_accuracy_thresh after epoch 4: 0.9972139000892639: 
04/27/2020 19:35:13 - INFO - root -   eval_roc_auc after epoch 4: 0.9968096203723649: 
04/27/2020 19:35:13 - INFO - root -   eval_fbeta after epoch 4: 0.9425551295280457: 
04/27/2020 19:35:13 - INFO - root -   lr after epoch 4: 1.4425359986282404e-05
04/27/2020 19:35:13 - INFO - root -   train_loss after epoch 4: 0.03893895296256661
04/27/2020 19:35:13 - INFO - root -   

04/27/2020 19:53:21 - INFO - root -   Running evaluation
04/27/2020 19:53:21 - INFO - root -     Num examples = 2176
04/27/2020 19:53:21 - INFO - root -     Batch size = 16


04/27/2020 19:55:11 - INFO - root -   eval_loss after epoch 5: 0.028918324024690426: 
04/27/2020 19:55:11 - INFO - root -   eval_accuracy_thresh after epoch 5: 0.9980037808418274: 
04/27/2020 19:55:11 - INFO - root -   eval_roc_auc after epoch 5: 0.9971317768575944: 
04/27/2020 19:55:11 - INFO - root -   eval_fbeta after epoch 5: 0.9623161554336548: 
04/27/2020 19:55:11 - INFO - root -   lr after epoch 5: 3.912491845372765e-06
04/27/2020 19:55:11 - INFO - root -   train_loss after epoch 5: 0.0329441800217062
04/27/2020 19:55:11 - INFO - root -   

04/27/2020 20:13:19 - INFO - root -   Running evaluation
04/27/2020 20:13:19 - INFO - root -     Num examples = 2176
04/27/2020 20:13:19 - INFO - root -     Batch size = 16


04/27/2020 20:15:09 - INFO - root -   eval_loss after epoch 6: 0.028500473852652836: 
04/27/2020 20:15:09 - INFO - root -   eval_accuracy_thresh after epoch 6: 0.9980037808418274: 
04/27/2020 20:15:09 - INFO - root -   eval_roc_auc after epoch 6: 0.997247313771853: 
04/27/2020 20:15:09 - INFO - root -   eval_fbeta after epoch 6: 0.9632353186607361: 
04/27/2020 20:15:09 - INFO - root -   lr after epoch 6: 0.0
04/27/2020 20:15:09 - INFO - root -   train_loss after epoch 6: 0.03135984920663759
04/27/2020 20:15:09 - INFO - root -   



(6528, 0.09323442643953432)

In [26]:
learner.validate()

04/27/2020 21:38:51 - INFO - root -   Running evaluation
04/27/2020 21:38:51 - INFO - root -     Num examples = 2176
04/27/2020 21:38:51 - INFO - root -     Batch size = 16


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


KeyboardInterrupt: ignored

In [1]:
learner.save_model()

NameError: ignored

In [0]:
learner.predict_batch(list(pd.read_csv('./drive/My Drive/Colab Notebooks/ap_product/predict.csv')['Text'].values))

In [0]:
from fast_bert.prediction import BertClassificationPredictor


In [25]:
predictorProduct = BertClassificationPredictor(
				model_path='./drive/My Drive/Colab Notebooks/ap_product/models/output/model_out',
				label_path=LABEL_PATH, # location for labels.csv file
				multi_label=True,
				model_type='xlnet',
				do_lower_case=True)


04/27/2020 22:52:31 - INFO - transformers.configuration_utils -   loading configuration file ./drive/My Drive/Colab Notebooks/ap_product/models/output/model_out/config.json
04/27/2020 22:52:31 - INFO - transformers.configuration_utils -   Model config XLNetConfig {
  "_num_labels": 32,
  "architectures": [
    "XLNetForMultiLabelSequenceClassification"
  ],
  "attn_type": "bi",
  "bad_words_ids": null,
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "decoder_start_token_id": null,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": false,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "

In [0]:
predictorProduct.predict_batch(list(pd.read_csv('./drive/My Drive/Colab Notebooks/ap_product/predict2.csv')['Text'].values))

In [26]:
df = pd.read_csv('./drive/My Drive/Colab Notebooks/ap_product/Argus_cleaned_all.csv')
df.head()

,Source,Unit,Frequency,Short Label,Include,Product,Geography,Long Label,Mnemonic,Favorite,Category,NLPText
0,Argus,NA,Daily,Canadian Syncrude cumulative no of trades trade month,Yes,None,None,Canadian Syncrude cumulative no of trades trade month,NaN,No,Crude North America Canada,Crude North America Canada Canadian Syncrude cumulative no of trades trade month NA
1,Argus,NA,Daily,Canadian Syncrude daily no of trades month,Yes,None,None,Canadian Syncrude daily no of trades month,NaN,No,Crude North America Canada,Crude North America Canada Canadian Syncrude daily no of trades month NA
2,Argus,NA,Daily,WCS cumulative no of trades trade month,Yes,None,None,WCS cumulative no of trades trade month,NaN,No,Crude North America Canada,Crude North America Canada WCS cumulative no of trades trade month NA
3,Argus,NA,Daily,WCS daily no of trades month,Yes,None,None,WCS daily no of trades month,NaN,No,Crude North America Canada,Crude North America Canada WCS daily no of trades month NA
4,Argus,NA,Daily,Normal butane Mt Belvieu Enterprise percentage of WTI,Yes,None,None,Normal butane Mt Belvieu Enterprise percentage of WTI,PA0015327,No,LPG Americas Butane,LPG Americas Butane Normal butane Mt Belvieu Enterprise percentage of WTI NA


In [27]:
for i, items in df.iterrows():
  predictedProduct = predictorProduct.predict(items['NLPText'])
  if i > 10:
    break
  print(predictedProduct)
  df.at[i, 'Product']= predictedProduct[0][0]

04/27/2020 22:53:06 - INFO - root -   Writing example 0 of 1
[(' Crude Oil', 0.9348597526550293), (' Natural Gas', 0.02887689508497715), (' Fuel Oil', 0.023671260103583336), (' Gasoline', 0.02177772857248783), (' Gasoil', 0.02074127271771431), (' Base Oil', 0.019691135734319687), (' Jet Fuel', 0.01951160654425621), (' Diesel', 0.018108507618308067), (' Phenol', 0.01746187172830105), (' Butane', 0.0169447623193264), (' Ethylene', 0.016864731907844543), (' Styrene', 0.016570698469877243), (' Benzene', 0.01636875607073307), (' PropaneButane', 0.016172928735613823), (' Propylene', 0.015831811353564262), (' Cyclohexane', 0.015338094905018806), (' Rig Count', 0.015211647376418114), (' Propane', 0.013240614905953407), (' Ehtylene', 0.013160612434148788), (' Naphtha', 0.012918935157358646), (' Isobutane', 0.01286645419895649), (' Butadiene', 0.0127991558983922), (' Ethane', 0.012570925988256931), (' VGO', 0.012149344198405743), (' Cumene', 0.01180944126099348), (' Coal', 0.011374994181096554),

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[(' Butane', 0.3793094754219055), (' Propane', 0.121246837079525), (' Natural Gas', 0.10907885432243347), (' PropaneButane', 0.06515522301197052), (' Isobutane', 0.06411774456501007), (' Fuel Oil', 0.05392221361398697), (' Ethylene', 0.04712262377142906), (' Butadiene', 0.045619066804647446), (' Naphtha', 0.04448724165558815), (' Ethane', 0.04083181545138359), (' Jet Fuel', 0.039094291627407074), (' Crude Oil', 0.03721371665596962), (' Phenol', 0.03425922244787216), (' Ehtylene', 0.03332330659031868), (' EthanePropane', 0.03056146204471588), (' Benzene', 0.02946353144943714), (' Styrene', 0.02920631878077984), (' Diesel', 0.027162451297044754), (' Coal', 0.02714468352496624), (' Propylene', 0.026148907840251923), (' Isobutylene', 0.026033291593194008), (' Mogas', 0.025886278599500656), (' VGO', 0.02464252896606922), (' Gasoil', 0.024234358221292496), (' Polybutadiene', 0.02369552105665207), (' Base Oil', 0.023385299369692802), (' Cyclohexane', 0.022127244621515274), (' Acrylonitrite', 

In [36]:
randomText = "Naphtha CIF Med Cargo"
predictedProduct = predictorProduct.predict(randomText)
print(predictedProduct)

04/27/2020 22:57:50 - INFO - root -   Writing example 0 of 1
[(' Naphtha', 0.4948664605617523), (' Jet Fuel', 0.12704402208328247), (' Base Oil', 0.07248331606388092), (' Butane', 0.0706823393702507), (' Crude Oil', 0.06918984651565552), (' Ethylene', 0.065948486328125), (' Coal', 0.06125534698367119), (' Diesel', 0.05268755927681923), (' Propane', 0.05066140741109848), (' Natural Gas', 0.049060262739658356), (' PropaneButane', 0.04886212572455406), (' Gasoline', 0.04537946730852127), (' Isobutylene', 0.04123910516500473), (' Phenol', 0.03929688036441803), (' EthanePropane', 0.03906182199716568), (' Fuel Oil', 0.03905410319566727), (' Ethane', 0.03833062946796417), (' Benzene', 0.034010060131549835), (' Isobutane', 0.033366162329912186), (' Styrene', 0.03190544620156288), (' Mogas', 0.031346920877695084), (' Propylene', 0.03075850009918213), (' Polybutadiene', 0.030395518988370895), (' Butadiene', 0.028619833290576935), (' Gasoil', 0.026450177654623985), (' Acrylonitrite', 0.0246388521

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
